# Langchain - RAG - Bot Using GoogleGenerativeAi

This project involves creating a Car AI Assistant capable of performing various tasks based on user commands. The assistant leverages a Large Language Model for performing these tasks. This is a step by step guide showing how various things are implemented throughout to improve the Code part by part.

[![MIT License](https://img.shields.io/badge/License-MIT-green.svg)](https://choosealicense.com/licenses/mit/)

In [ ]:
!pip install -U langchain-google-genai
!pip install -U langchain
!pip install -U langchain_community
!pip install -U huggingface_hub
!pip install -U transformers
!pip install -U accelerate
!pip install -U google-generativeai
!pip install -U fuzzywuzzy
!pip install -U duckduckgo-search
!pip install -U sentence-transformers
!pip install -U pypdf
!pip install -U chromadb
!pip install -U faiss-cpu
!pip install -U pyngrok
!pip install -U fastapi uvicorn


# Using Bert

This method was the initial thinking but later was transformed into a more stable form using langchain and llm integration



In [ ]:
from transformers import pipeline
import requests
from fuzzywuzzy import fuzz

# Initialize the NLP model
nlp = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# User preferences
user_preferences = {
    "music": "rock",
    "navigation": "shortest route",
    "climate": "cool",
}

# API key for weather service
WEATHER_API_KEY = "39f58c559276ac165d4c6facf36dab3e"

def process_command(command):
    context = """
    Your car's AI assistant can perform various tasks:
    1. Play music of different genres
    2. Navigate to destinations using various routing options
    3. Adjust climate settings in the car
    4. Provide weather information for different cities
    """

    try:
        action = nlp(question=command, context=context)['answer']
        print(f"Interpreted action: {action}")

        if fuzz.partial_ratio("play music", action.lower()) > 70:
            response = f"Playing {user_preferences['music']} music for you. Enjoy!"
        elif fuzz.partial_ratio("navigate", action.lower()) > 70:
            response = f"Setting up navigation using your preferred {user_preferences['navigation']}. Where would you like to go?"
        elif fuzz.partial_ratio("climate", action.lower()) > 70:
            response = f"Adjusting the climate to your preferred {user_preferences['climate']} setting. Let me know if you need any changes."
        elif fuzz.partial_ratio("weather", action.lower()) > 70:
            response = "I can provide weather information. Which city would you like to know about?"
        else:
            response = "I'm not sure I understood that command. Could you please rephrase it?"
    except Exception as e:
        print(f"Error: {e}")
        response = "I encountered an error while processing your request. Could you try again?"

    return response

def get_weather(city='San Francisco'):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        weather_data = {
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"],
            "icon": data["weather"][0]["icon"]
        }
        return weather_data
    else:
        return {"error": "City not found"}

# Example usage
if __name__ == "__main__":
    print("Welcome to your car's AI assistant! How can I help you today?")
    while True:
        user_input = input("Your command: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye! Have a great day!")
            break

        response = process_command(user_input)
        print(response)

        if "weather" in response.lower():
            city = input("Enter the city name: ").strip()
            weather_info = get_weather(city)
            if "error" in weather_info:
                print(f"I'm sorry, I couldn't find weather information for {city}.")
            else:
                print(f"Here's the weather in {city}:")
                print(f"Temperature: {weather_info['temperature']}°C")
                print(f"Description: {weather_info['description']}")
                print(f"Icon code: {weather_info['icon']}")


# First Langchain Integration

The assistant can process commands related to music, navigation, climate control, and weather updates. By utilizing LangChain's advanced conversational capabilities and integrating with a weather API, the assistant offers a more intelligent and interactive user experience compared to the previous implementation.



In [ ]:
import os
from langchain.llms import GooglePalm
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import requests
from fuzzywuzzy import fuzz

# Set up the API key
os.environ['GOOGLE_API_KEY'] = 'YOUR-API-KEY'

# Initialize the language model
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY)

# User preferences
user_preferences = {
    "music": "rock",
    "navigation": "shortest route",
    "climate": "cool",
}

# API key for weather service
WEATHER_API_KEY = "KEY"

# Define the prompt template for car assistant
car_template = """
You are an AI assistant for a smart car. You can help with playing music, navigation, climate control, and providing weather information.
Current user preferences:
- Music: {music_preference}
- Navigation: {navigation_preference}
- Climate: {climate_preference}

Focus on understanding and responding to car-related commands and queries.
If multiple actions are requested, address them in the order mentioned.

Current conversation:
{history}
Human: {input}
AI:
"""

car_prompt = PromptTemplate(
    input_variables=["history", "input", "music_preference", "navigation_preference", "climate_preference"],
    template=car_template
)

# Initialize the memory
memory = ConversationBufferMemory(input_key="input", memory_key="history")

# Initialize the LLMChain for car assistant
car_chain = LLMChain(
    llm=llm,
    prompt=car_prompt,
    memory=memory,
    verbose=True
)

def process_command(command):
    try:
        response = car_chain.predict(
            input=command,
            music_preference=user_preferences["music"],
            navigation_preference=user_preferences["navigation"],
            climate_preference=user_preferences["climate"]
        )
        return response
    except Exception as e:
        print(f"Error: {e}")
        return "I encountered an error while processing your request. Could you try again?"

def get_weather(city='San Francisco'):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        weather_data = {
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"],
            "icon": data["weather"][0]["icon"]
        }
        return weather_data
    else:
        return {"error": "City not found"}

# Example usage
if __name__ == "__main__":
    print("Welcome to your car's AI assistant! How can I help you today?")
    while True:
        user_input = input("Your command: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye! Have a great day!")
            break

        response = process_command(user_input)
        print("AI:", response)

        if "weather" in response.lower() and "which city" in response.lower():
            city = input("Enter the city name: ").strip()
            weather_info = get_weather(city)
            if "error" in weather_info:
                print(f"I'm sorry, I couldn't find weather information for {city}.")
            else:
                print(f"Here's the weather in {city}:")
                print(f"Temperature: {weather_info['temperature']}°C")
                print(f"Description: {weather_info['description']}")
                print(f"Icon code: {weather_info['icon']}")

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Welcome to your car's AI assistant! How can I help you today?
Your command: i feel cold


> Entering new LLMChain chain...
Prompt after formatting:

You are an AI assistant for a smart car. You can help with playing music, navigation, climate control, and providing weather information.
Current user preferences:
- Music: rock
- Navigation: shortest route
- Climate: cool

Focus on understanding and responding to car-related commands and queries.
If multiple actions are requested, address them in the order mentioned.

Current conversation:

Human: i feel cold
AI:


> Finished chain.
AI: I can adjust the temperature for you. Would you like it to be warmer? 

Your command: on my way to home play some music


> Entering new LLMChain chain...
Prompt after formatting:

You are an AI assistant for a smart car. You can help with playing music, navigation, climate control, and providing weather information.
Current user preferences:
- Music: rock
- Navigation: shortest route
- Climate: cool

Focu


# Adding Agents

 The use of LangChain's tool-based architecture allows the assistant to manage complex commands more effectively and provide more accurate and helpful responses compared to the previous implementation.




In [ ]:
import os
from langchain.llms import GooglePalm
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import requests
from langchain.tools import DuckDuckGoSearchRun

# Set up the API key
os.environ['GOOGLE_API_KEY'] = 'KEY'

# Initialize the language model
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key='KEY')

# User preferences
user_preferences = {
    "music": "rock",
    "navigation": "shortest route",
    "climate": "cool",
}

# API key for weather service
WEATHER_API_KEY = "KEY"

def get_weather(city='San Francisco'):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        weather_data = {
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"],
            "icon": data["weather"][0]["icon"]
        }
        return f"Weather in {city}: Temperature: {weather_data['temperature']}°C, Description: {weather_data['description']}"
    else:
        return f"Could not find weather information for {city}."

def play_music(genre=None):
    genre = genre or user_preferences["music"]
    return f"Playing {genre} music."

def navigate(destination):
    return f"Navigating to {destination} using {user_preferences['navigation']}."

def adjust_climate(setting=None):
    setting = setting or user_preferences["climate"]
    return f"Adjusting climate to {setting}."

# Initialize the search tool
search = DuckDuckGoSearchRun()

# Define the tools
tools = [
    Tool(
        name="Weather",
        func=get_weather,
        description="Get the current weather in a given city."
    ),
    Tool(
        name="Music",
        func=play_music,
        description="Play music of a specified genre."
    ),
    Tool(
        name="Navigation",
        func=navigate,
        description="Navigate to a specified destination."
    ),
    Tool(
        name="Climate",
        func=adjust_climate,
        description="Adjust the car's climate settings."
    ),
    Tool(
        name="Search",
        func=search.run,
        description="Search the internet for general information."
    )
]

# Initialize the memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Custom prompt template
custom_prompt = PromptTemplate(
    template="""You are an AI assistant for a smart car. You can help with playing music, navigation, climate control, providing weather information, and searching for general information.
Current user preferences:
- Music: {music_preference}
- Navigation: {navigation_preference}
- Climate: {climate_preference}

Use the provided tools to assist the user with their requests. If multiple actions are requested, address them in the order mentioned.
Always strive to give helpful and accurate responses.

{chat_history}
Human: {input}
AI: Let's approach this step-by-step:
""",
    input_variables=["chat_history", "input", "music_preference", "navigation_preference", "climate_preference"]
)

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent="chat-conversational-react-description",
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={"system_message": custom_prompt.format(
        music_preference=user_preferences["music"],
        navigation_preference=user_preferences["navigation"],
        climate_preference=user_preferences["climate"],
        chat_history="",
        input=""
    )}
)

# Main interaction loop
if __name__ == "__main__":
    print("Welcome to your car's AI assistant! How can I help you today?")
    while True:
        user_input = input("Your command: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye! Have a great day!")
            break

        try:
            response = agent.run(user_input)
            print("AI:", response)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("I apologize for the inconvenience. Is there anything else I can help you with?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


Welcome to your car's AI assistant! How can I help you today?
Your command: play music while going to work


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Navigation",
    "action_input": "Navigate to work"
}
```
Observation: Navigating to Navigate to work using shortest route.
Thought:```json
{
    "action": "Music",
    "action_input": "Play rock music"
}
```
Observation: Playing Play rock music music.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Okay, I am now playing rock music on our way to work."
}
```

> Finished chain.
AI: Okay, I am now playing rock music on our way to work.
Your command: what route should i take today on my way to home


> Entering new AgentExecutor chain...


```json
{
    "action": "Navigation",
    "action_input": "Home"
}
```
Observation: Navigating to Home using shortest route.
Thought:

```json
{
"action": "Final Answer",
"action_input": "You are currently being navigated home using the shortest route."
}
```


> Finished chain.
AI: You are currently being navigated home using the shortest route.
Your command: i am feeling too hot


> Entering new AgentExecutor chain...
```json
{
    "action": "Climate",
    "action_input": "Decrease cabin temperature"
}
```
Observation: Adjusting climate to Decrease cabin temperature.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Okay, I have adjusted the temperature to be cooler."
}
```


> Finished chain.
AI: Okay, I have adjusted the temperature to be cooler.
Your command: whats the weather like today


> Entering new AgentExecutor chain...
```json
{
    "action": "Weather",
    "action_input": "current location"
}
```
Observation: Could not find weather information for current location.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "I am sorry, I do not have the weather information for


## Added Vector Databases

Enhances the smart car AI assistant by incorporating vector databases and additional functionalities. The integration of vector databases enables efficient and accurate retrieval of information from the car manual and other large documents, improving the overall user experience. Here are the key improvements and additions:

Vector Database for Car Manual:



- Loading and Caching: The car manual is loaded asynchronously and stored in a vector database (FAISS) for efficient retrieval.
Querying the Manual: Users can query the car manual for specific information, and the system retrieves the most relevant sections using vector embeddings.
New Tools and Functionalities:

- Trip Planning: A new tool for planning trips based on user queries, generating a structured response with destinations, duration, and activities.
Expanded Toolset: The assistant now includes tools for playing music, navigation, climate control, weather information, internet searches, and car manual queries.




In [ ]:
import os
import pickle
from threading import Thread
from langchain.llms import GooglePalm
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from pydantic import BaseModel, Field
from typing import List
import requests
import json
from langchain.vectorstores import FAISS

# Set up the API key
os.environ['GOOGLE_API_KEY'] = 'KEY'

# Initialize the language model
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key='KEY')

# User preferences
user_preferences = {
    "music": "rock",
    "navigation": "shortest route",
    "climate": "cool",
}

# API key for weather service
WEATHER_API_KEY = "KEY"

# Structured output parser for weather
weather_schema = ResponseSchema(name="weather", description="Weather information including temperature and description")
weather_parser = StructuredOutputParser.from_response_schemas([weather_schema])

def get_weather(city='San Francisco'):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        weather_data = {
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"]
        }
        return weather_parser.parse(f"Weather in {city}: Temperature: {weather_data['temperature']}°C, Description: {weather_data['description']}")
    else:
        return f"Could not find weather information for {city}."

def play_music(genre=None):
    genre = genre or user_preferences["music"]
    return f"Playing {genre} music."

def navigate(destination):
    return f"Navigating to {destination} using {user_preferences['navigation']}."

def adjust_climate(setting=None):
    setting = setting or user_preferences["climate"]
    return f"Adjusting climate to {setting}."

# Trip planning tool
class TripPlan(BaseModel):
    destinations: List[str] = Field(description="List of destinations to visit")
    duration: int = Field(description="Trip duration in days")
    activities: List[str] = Field(description="List of planned activities")

def plan_trip(query: str) -> str:
    trip_chain = LLMChain(llm=llm, prompt=PromptTemplate(
        input_variables=["query"],
        template="Plan a trip based on this query: {query}. Provide a list of destinations, trip duration in days, and planned activities. Format your response as a JSON object with keys 'destinations' (list of strings), 'duration' (integer), and 'activities' (list of strings)."
    ))

    try:
        result = trip_chain.run(query)
        trip_data = json.loads(result)
        trip_plan = TripPlan(**trip_data)
        return f"Trip Plan: Destinations: {', '.join(trip_plan.destinations)}, Duration: {trip_plan.duration} days, Activities: {', '.join(trip_plan.activities)}"
    except Exception as e:
        return f"I apologize, I couldn't create a proper trip plan. Here's the raw output: {result}"

# Global variable for car manual QA
car_manual_qa = None

def load_car_manual():
    cache_path = "car_manual_cache.pkl"
    if os.path.exists(cache_path):
        print("Loading car manual from cache...")
        with open(cache_path, "rb") as f:
            cache_data = pickle.load(f)

        texts = cache_data["texts"]
        embeddings = cache_data["embeddings"]

        # Recreate the vectorstore
        vectorstore = FAISS.from_embeddings(text_embeddings=zip([text.page_content for text in texts], embeddings), embedding=HuggingFaceEmbeddings())

        return RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
    else:
        print("Error: Car manual cache not found. Please run the caching script first.")
        return None

def load_manual_async():
    global car_manual_qa
    try:
        car_manual_qa = load_car_manual()
        if car_manual_qa:
            print("Car manual fully loaded and ready!")
        else:
            print("Failed to load car manual.")
    except Exception as e:
        print(f"Error loading car manual: {str(e)}")
        car_manual_qa = None

# Start loading the car manual in a separate thread
Thread(target=load_manual_async, daemon=True).start()

def car_manual_query(query):
    if car_manual_qa:
        try:
            return car_manual_qa.run(query)
        except Exception as e:
            return f"An error occurred while querying the car manual: {str(e)}"
    else:
        return "The car manual is still loading or encountered an error during loading. Please try again later."

# Initialize the search tool
search = DuckDuckGoSearchRun()

# Define the tools
tools = [
    Tool(name="Weather", func=get_weather, description="Get the current weather in a given city."),
    Tool(name="Music", func=play_music, description="Play music of a specified genre."),
    Tool(name="Navigation", func=navigate, description="Navigate to a specified destination."),
    Tool(name="Climate", func=adjust_climate, description="Adjust the car's climate settings."),
    Tool(name="Search", func=search.run, description="Search the internet for general information."),
    Tool(name="TripPlanner", func=plan_trip, description="Plan a trip with destinations, duration, and activities."),
    Tool(name="CarManual", func=car_manual_query, description="Get information from the car manual.")
]

# Initialize the memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Custom prompt template
custom_prompt = PromptTemplate(
    template="""You are an advanced AI assistant for a smart car. You can help with various tasks including playing music, navigation, climate control, weather information, trip planning, car manual inquiries, and general information searches.
Current user preferences:
- Music: {music_preference}
- Navigation: {navigation_preference}
- Climate: {climate_preference}

Use the provided tools to assist the user with their requests. If multiple actions are requested, address them in the order mentioned.
For complex queries, think step-by-step and explain your reasoning.

{chat_history}
Human: {input}
AI: Let's approach this step-by-step:
""",
    input_variables=["chat_history", "input", "music_preference", "navigation_preference", "climate_preference"]
)

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={"system_message": custom_prompt.format(
        music_preference=user_preferences["music"],
        navigation_preference=user_preferences["navigation"],
        climate_preference=user_preferences["climate"],
        chat_history="",
        input=""
    )}
)

# Main interaction loop
if __name__ == "__main__":
    print("Welcome to your advanced car's AI assistant! How can I help you today?")
    print("(The car manual is being loaded in the background and will be available shortly.)")
    while True:
        user_input = input("Your command: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye! Have a great day!")
            break

        try:
            response = agent.run(user_input)
            print("AI:", response)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("I apologize for the inconvenience. Is there anything else I can help you with?")

Loading car manual from cache...
Welcome to your advanced car's AI assistant! How can I help you today?
(The car manual is being loaded in the background and will be available shortly.)
Car manual fully loaded and ready!
Your command: what is my car name


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "I am sorry, I do not have access to your car's information and therefore do not know your car's name."
}
```

> Finished chain.
AI: I am sorry, I do not have access to your car's information and therefore do not know your car's name.
Your command: what is my car name form my carmanual


> Entering new AgentExecutor chain...
```json
{
    "action": "CarManual",
    "action_input": "Find car name"
}
```
Observation: The provided text does not contain the car's name. 

Thought:```json
{
"action": "Final Answer",
"action_input": "Unfortunately, I still don't have access to that information. The car manual did not provide your car's name."
}



> Entering new AgentExecutor chain...


```json
{
    "action": "CarManual",
    "action_input": "What are the featured items in the dashboard?"
}
```


Observation: The dashboard, also known as the instrument cluster, features:

* **Tachometer**
* **Speedometer** (available as both analog and digital displays)
* **Fuel Gauge**
* **Engine Coolant Temperature Gauge**
* **Warning and Indicator Lights**
* **Cluster Display** (including Trip Computer)
* **Trip Mode/Reset Button**
* **Gear Position Pop-Up** (optional)
* **Service Interval Indicator**
* **Elapsed Time** (if equipped) 

The exact features and layout may vary depending on the specific vehicle model and trim level ("Type A", "Type B", or "Type C"). 

Thought:```json
{
    "action": "Final Answer",
    "action_input": "The dashboard, also known as the instrument cluster, features a tachometer, speedometer, fuel gauge, engine coolant temperature gauge, warning and indicator lights, a cluster display which includes the trip computer, a trip mode/reset button, a gear position pop-up, a service interval indicator, and elapsed time."
}
```

> Finished chain.
AI: The dashboard, also 


# Caching The Pdf

This script caches the car manual to enhance retrieval efficiency by processing and embedding the manual's content.

In [ ]:
import os
import pickle
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

def cache_car_manual(pdf_path, cache_path="car_manual_cache.pkl"):
    print("Processing car manual. This may take a while...")
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings()
    embedded_texts = [embeddings.embed_query(text.page_content) for text in texts]

    cache_data = {
        "texts": texts,
        "embeddings": embedded_texts
    }

    print("Saving car manual to cache...")
    with open(cache_path, "wb") as f:
        pickle.dump(cache_data, f)

    print(f"Car manual cached successfully to {cache_path}")

if __name__ == "__main__":
    pdf_path = "grand-i10-nios.pdf"  # Update this path to your car manual PDF
    cache_path = "car_manual_cache.pkl"

    if os.path.exists(pdf_path):
        cache_car_manual(pdf_path, cache_path)
    else:
        print(f"Error: Car manual PDF not found at {pdf_path}")

Processing car manual. This may take a while...
Saving car manual to cache...
Car manual cached successfully to car_manual_cache.pkl


### Adding the ngrok auth token which is needed to run our Streamlit app

In [ ]:
!ngrok authtoken YOUR-AUTH-TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml



# Exposing The Endpoint

This code sets up a FastAPI server to provide API access to an advanced AI assistant for a smart car. Here's how it facilitates API-based interaction:

### FastAPI Server Setup:

- FastAPI Framework: The FastAPI framework is used to create a web server that handles HTTP requests and serves responses.
CORS Middleware: Configured to allow cross-origin requests from specific domains (e.g., localhost and deployed front-end applications).
API Endpoint:

- /chat Endpoint: A POST request to /chat accepts user messages. This endpoint uses the AI agent to process the message and generate a response.
Pydantic Model: The UserMessage model ensures that the incoming requests have a message field, enforcing data validation.

### Ngrok Integration:

- Public URL Exposure: Ngrok is used to expose the local FastAPI server to the public internet, allowing external applications to interact with the API. The public URL provided by ngrok can be used for remote access and testing.

In [ ]:
import os
import pickle
from threading import Thread
from langchain.llms import GooglePalm
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from pydantic import BaseModel, Field
from pyngrok import ngrok
from typing import List
import requests
import json
import uvicorn
from langchain.vectorstores import FAISS
from fastapi import FastAPI, HTTPException
import asyncio
from concurrent.futures import ThreadPoolExecutor
from fastapi.middleware.cors import CORSMiddleware

# Set up the API key
os.environ['GOOGLE_API_KEY'] = 'Key'

# Initialize the language model
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key='Key')

# User preferences
user_preferences = {
    "music": "rock",
    "navigation": "shortest route",
    "climate": "cool",
}

# API key for weather service
WEATHER_API_KEY = "Key"

# Structured output parser for weather
weather_schema = ResponseSchema(name="weather", description="Weather information including temperature and description")
weather_parser = StructuredOutputParser.from_response_schemas([weather_schema])

def get_weather(city='San Francisco'):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        weather_data = {
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"]
        }
        return weather_parser.parse(f"Weather in {city}: Temperature: {weather_data['temperature']}°C, Description: {weather_data['description']}")
    else:
        return f"Could not find weather information for {city}."

def play_music(genre=None):
    genre = genre or user_preferences["music"]
    return f"Playing {genre} music."

def navigate(destination):
    return f"Navigating to {destination} using {user_preferences['navigation']}."

def adjust_climate(setting=None):
    setting = setting or user_preferences["climate"]
    return f"Adjusting climate to {setting}."

# Trip planning tool
class TripPlan(BaseModel):
    destinations: List[str] = Field(description="List of destinations to visit")
    duration: int = Field(description="Trip duration in days")
    activities: List[str] = Field(description="List of planned activities")

def plan_trip(query: str) -> str:
    trip_chain = LLMChain(llm=llm, prompt=PromptTemplate(
        input_variables=["query"],
        template="Plan a trip based on this query: {query}. Provide a list of destinations, trip duration in days, and planned activities. Format your response as a JSON object with keys 'destinations' (list of strings), 'duration' (integer), and 'activities' (list of strings)."
    ))

    try:
        result = trip_chain.run(query)
        trip_data = json.loads(result)
        trip_plan = TripPlan(**trip_data)
        return f"Trip Plan: Destinations: {', '.join(trip_plan.destinations)}, Duration: {trip_plan.duration} days, Activities: {', '.join(trip_plan.activities)}"
    except Exception as e:
        return f"I apologize, I couldn't create a proper trip plan. Here's the raw output: {result}"

# Global variable for car manual QA
car_manual_qa = None

def load_car_manual():
    cache_path = "car_manual_cache.pkl"
    if os.path.exists(cache_path):
        print("Loading car manual from cache...")
        with open(cache_path, "rb") as f:
            cache_data = pickle.load(f)

        texts = cache_data["texts"]
        embeddings = cache_data["embeddings"]

        # Recreate the vectorstore
        vectorstore = FAISS.from_embeddings(text_embeddings=zip([text.page_content for text in texts], embeddings), embedding=HuggingFaceEmbeddings())

        return RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
    else:
        print("Error: Car manual cache not found. Please run the caching script first.")
        return None

def load_manual_async():
    global car_manual_qa
    try:
        car_manual_qa = load_car_manual()
        if car_manual_qa:
            print("Car manual fully loaded and ready!")
        else:
            print("Failed to load car manual.")
    except Exception as e:
        print(f"Error loading car manual: {str(e)}")
        car_manual_qa = None

# Start loading the car manual in a separate thread
Thread(target=load_manual_async, daemon=True).start()

def car_manual_query(query):
    if car_manual_qa:
        try:
            return car_manual_qa.run(query)
        except Exception as e:
            return f"An error occurred while querying the car manual: {str(e)}"
    else:
        return "The car manual is still loading or encountered an error during loading. Please try again later."

# Initialize the search tool
search = DuckDuckGoSearchRun()

# Define the tools
tools = [
    Tool(name="Weather", func=get_weather, description="Get the current weather in a given city."),
    Tool(name="Music", func=play_music, description="Play music of a specified genre."),
    Tool(name="Navigation", func=navigate, description="Navigate to a specified destination."),
    Tool(name="Climate", func=adjust_climate, description="Adjust the car's climate settings."),
    Tool(name="Search", func=search.run, description="Search the internet for general information."),
    Tool(name="TripPlanner", func=plan_trip, description="Plan a trip with destinations, duration, and activities."),
    Tool(name="CarManual", func=car_manual_query, description="Get information from the car manual.")
]

# Initialize the memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Custom prompt template
custom_prompt = PromptTemplate(
    template="""You are an advanced AI assistant for a smart car. You can help with various tasks including playing music, navigation, climate control, weather information, trip planning, car manual inquiries, and general information searches.
Current user preferences:
- Music: {music_preference}
- Navigation: {navigation_preference}
- Climate: {climate_preference}

Use the provided tools to assist the user with their requests. If multiple actions are requested, address them in the order mentioned.
For complex queries, think step-by-step and explain your reasoning.

{chat_history}
Human: {input}
AI: Let's approach this step-by-step:
""",
    input_variables=["chat_history", "input", "music_preference", "navigation_preference", "climate_preference"]
)

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={"system_message": custom_prompt.format(
        music_preference=user_preferences["music"],
        navigation_preference=user_preferences["navigation"],
        climate_preference=user_preferences["climate"],
        chat_history="",
        input=""
    )}
)

# Initialize FastAPI app
app = FastAPI()

origins = [
    "http://localhost:3000",
    "https://car-dashboard-ivory.vercel.app"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Pydantic model for the request
class UserMessage(BaseModel):
    message: str

# Endpoint to handle user messages
@app.post("/chat")
async def chat(user_message: UserMessage):
    try:
        response = agent.run(user_message.message)
        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Main function to run the FastAPI server with ngrok
def run():
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)

    config = uvicorn.Config(app, port=8000, loop="asyncio")
    server = uvicorn.Server(config)

    with ThreadPoolExecutor() as executor:
        executor.submit(server.run)

    # Keep the main thread alive
    while True:
        try:
            asyncio.sleep(1)
        except KeyboardInterrupt:
            break

if __name__ == "__main__":
    print("Starting the AI assistant server...")
    print("The car manual is being loaded in the background and will be available shortly.")

    # Start loading the car manual in a separate thread
    Thread(target=load_manual_async, daemon=True).start()

    # Run the FastAPI server with ngrok
    run()

# Logging

The script provided is designed to interact with a user via a console-based AI assistant, and it includes functionality for logging responses and user interactions. Here's how it manages and saves data for future use:

### Interaction Logging:

- Conversation Data: Each interaction with the AI assistant is logged, including user inputs, AI responses, tool usage, timestamps, processing times, and any errors.
- Session Management: A unique session ID is generated for each session, and interactions are grouped under this session ID.

### CSV Conversion for Fine-Tuning:

- Data Extraction: After logging interactions in JSON format, the responses and related data can be extracted and converted to a CSV file.
- CSV Benefits: CSV files are widely used for data manipulation and analysis. They are useful for preparing datasets for model fine-tuning and other data-driven tasks.
Future Fine-Tuning: The saved CSV data can be used to retrain or fine-tune the AI model, improving its performance based on real-world interactions and feedback.

In [ ]:
import os
import pickle
import json
from datetime import datetime
from threading import Thread
from langchain.llms import GooglePalm
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from pydantic import BaseModel, Field
from typing import List
import requests
from langchain.vectorstores import FAISS

# Set up the API key
os.environ['GOOGLE_API_KEY'] = 'KEY'

# Initialize the language model
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key='KEY')

# User preferences
user_preferences = {
    "music": "rock",
    "navigation": "shortest route",
    "climate": "cool",
}

# API key for weather service
WEATHER_API_KEY = "KEY"

# Structured output parser for weather
weather_schema = ResponseSchema(name="weather", description="Weather information including temperature and description")
weather_parser = StructuredOutputParser.from_response_schemas([weather_schema])

def get_weather(city='San Francisco'):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        weather_data = {
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"]
        }
        return weather_parser.parse(f"Weather in {city}: Temperature: {weather_data['temperature']}°C, Description: {weather_data['description']}")
    else:
        return f"Could not find weather information for {city}."

def play_music(genre=None):
    genre = genre or user_preferences["music"]
    return f"Playing {genre} music."

def navigate(destination):
    return f"Navigating to {destination} using {user_preferences['navigation']}."

def adjust_climate(setting=None):
    setting = setting or user_preferences["climate"]
    return f"Adjusting climate to {setting}."

# Trip planning tool
class TripPlan(BaseModel):
    destinations: List[str] = Field(description="List of destinations to visit")
    duration: int = Field(description="Trip duration in days")
    activities: List[str] = Field(description="List of planned activities")

def plan_trip(query: str) -> str:
    trip_chain = LLMChain(llm=llm, prompt=PromptTemplate(
        input_variables=["query"],
        template="Plan a trip based on this query: {query}. Provide a list of destinations, trip duration in days, and planned activities. Format your response as a JSON object with keys 'destinations' (list of strings), 'duration' (integer), and 'activities' (list of strings)."
    ))

    try:
        result = trip_chain.run(query)
        trip_data = json.loads(result)
        trip_plan = TripPlan(**trip_data)
        return f"Trip Plan: Destinations: {', '.join(trip_plan.destinations)}, Duration: {trip_plan.duration} days, Activities: {', '.join(trip_plan.activities)}"
    except Exception as e:
        return f"I apologize, I couldn't create a proper trip plan. Here's the raw output: {result}"

# Global variable for car manual QA
car_manual_qa = None

def load_car_manual():
    cache_path = "car_manual_cache.pkl"
    if os.path.exists(cache_path):
        print("Loading car manual from cache...")
        with open(cache_path, "rb") as f:
            cache_data = pickle.load(f)

        texts = cache_data["texts"]
        embeddings = cache_data["embeddings"]

        # Recreate the vectorstore
        vectorstore = FAISS.from_embeddings(text_embeddings=zip([text.page_content for text in texts], embeddings), embedding=HuggingFaceEmbeddings())

        return RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
    else:
        print("Error: Car manual cache not found. Please run the caching script first.")
        return None

def load_manual_async():
    global car_manual_qa
    try:
        car_manual_qa = load_car_manual()
        if car_manual_qa:
            print("Car manual fully loaded and ready!")
        else:
            print("Failed to load car manual.")
    except Exception as e:
        print(f"Error loading car manual: {str(e)}")
        car_manual_qa = None

# Start loading the car manual in a separate thread
Thread(target=load_manual_async, daemon=True).start()

def car_manual_query(query):
    if car_manual_qa:
        try:
            return car_manual_qa.run(query)
        except Exception as e:
            return f"An error occurred while querying the car manual: {str(e)}"
    else:
        return "The car manual is still loading or encountered an error during loading. Please try again later."

# Initialize the search tool
search = DuckDuckGoSearchRun()

# Define the tools
tools = [
    Tool(name="Weather", func=get_weather, description="Get the current weather in a given city."),
    Tool(name="Music", func=play_music, description="Play music of a specified genre."),
    Tool(name="Navigation", func=navigate, description="Navigate to a specified destination."),
    Tool(name="Climate", func=adjust_climate, description="Adjust the car's climate settings."),
    Tool(name="Search", func=search.run, description="Search the internet for general information."),
    Tool(name="TripPlanner", func=plan_trip, description="Plan a trip with destinations, duration, and activities."),
    Tool(name="CarManual", func=car_manual_query, description="Get information from the car manual.")
]

# Initialize the memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Custom prompt template
custom_prompt = PromptTemplate(
    template="""You are an advanced AI assistant for a smart car. You can help with various tasks including playing music, navigation, climate control, weather information, trip planning, car manual inquiries, and general information searches.
Current user preferences:
- Music: {music_preference}
- Navigation: {navigation_preference}
- Climate: {climate_preference}

Use the provided tools to assist the user with their requests. If multiple actions are requested, address them in the order mentioned.
For complex queries, think step-by-step and explain your reasoning.

{chat_history}
Human: {input}
AI: Let's approach this step-by-step:
""",
    input_variables=["chat_history", "input", "music_preference", "navigation_preference", "climate_preference"]
)

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={"system_message": custom_prompt.format(
        music_preference=user_preferences["music"],
        navigation_preference=user_preferences["navigation"],
        climate_preference=user_preferences["climate"],
        chat_history="",
        input=""
    )}
)

# Initialize or load existing conversation data
try:
    with open('conversation_data.json', 'r') as f:
        conversation_data = json.load(f)
except FileNotFoundError:
    conversation_data = []

# Function to get current user context
def get_user_context():
    return {
        "music_preference": user_preferences["music"],
        "navigation_preference": user_preferences["navigation"],
        "climate_preference": user_preferences["climate"],
        # Add any other relevant context here
    }

# Function to log tool usage
def log_tool_usage(tool_name, input_data, output_data):
    return {
        "tool_name": tool_name,
        "input": input_data,
        "output": output_data,
        "timestamp": datetime.now().isoformat()
    }

# Wrapper function for tools to log their usage
def tool_wrapper(tool):
    original_func = tool.func
    def wrapped_func(*args, **kwargs):
        result = original_func(*args, **kwargs)
        log_entry = log_tool_usage(tool.name, args[0] if args else kwargs, result)
        conversation_data[-1]["tool_usage"].append(log_entry)
        return result
    tool.func = wrapped_func
    return tool

# Wrap all tools with the logging function
tools = [tool_wrapper(tool) for tool in tools]

# Main interaction loop
if __name__ == "__main__":
    print("Welcome to your advanced car's AI assistant! How can I help you today?")
    print("(The car manual is being loaded in the background and will be available shortly.)")

    session_start_time = datetime.now().isoformat()
    session_id = f"session_{session_start_time}"

    while True:
        user_input = input("Your command: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye! Have a great day!")
            break

        try:
            # Start timing the response
            start_time = datetime.now()

            # Create a new entry for this interaction
            current_interaction = {
                "session_id": session_id,
                "timestamp": start_time.isoformat(),
                "user_input": user_input,
                "user_context": get_user_context(),
                "tool_usage": [],
                "processing_time": None,
                "ai_response": None,
                "error": None
            }

            # Add the new interaction to the conversation data
            conversation_data.append(current_interaction)

            response = agent.run(user_input)
            print("AI:", response)

            # Calculate processing time
            end_time = datetime.now()
            processing_time = (end_time - start_time).total_seconds()

            # Update the current interaction with the response and processing time
            current_interaction["ai_response"] = response
            current_interaction["processing_time"] = processing_time

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("I apologize for the inconvenience. Is there anything else I can help you with?")

            # Update the current interaction with error information
            current_interaction["error"] = str(e)

        # Save to file after each interaction
        with open('conversation_data.json', 'w') as f:
            json.dump(conversation_data, f, indent=2)

    # Log session end time
    session_end_time = datetime.now().isoformat()
    conversation_data.append({
        "session_id": session_id,
        "session_end_time": session_end_time
    })

    # Final save to file
    with open('conversation_data.json', 'w') as f:
        json.dump(conversation_data, f, indent=2)

Error: Car manual cache not found. Please run the caching script first.
Failed to load car manual.
Welcome to your advanced car's AI assistant! How can I help you today?
(The car manual is being loaded in the background and will be available shortly.)
Your command: hello


> Entering new AgentExecutor chain...
```json
{
"action": "Final Answer",
"action_input": "Hello! How can I assist you today?"
}
```

> Finished chain.
AI: Hello! How can I assist you today?
Your command: what is the weather today


> Entering new AgentExecutor chain...
```json
{
    "action": "Weather",
    "action_input": "current location"
}
```
Observation: Could not find weather information for current location.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "I'm sorry, I do not have the weather information for your current location."
}
```

> Finished chain.
AI: I'm sorry, I do not have the weather information for your current location.
Your command: i want to hear ro some music


> Enterin

```json
{
    "action": "Navigation",
    "action_input": "Home"
}
```
Observation: Navigating to Home using shortest route.
Thought:

```json
{
    "action": "Search",
    "action_input": "travel places"
}
```


Observation: Paris. #1 in World's Best Places to Visit for 2023-2024. France's magnetic City of Light is a perennial tourist destination, drawing visitors with its iconic attractions, like the Eiffel Tower and ... Glacier National Park. #1 in Best Places to Visit in the USA for 2023-2024. Glaciers are the main draw of this Montana national park, but its more than 700 lakes, two mountain ranges and multiple ... What are the best places to visit in the world? Time magazine just revealed its annual World's Greatest Places List, which highlights 100 amazing places around the globe—including hotels ... Our guidebooks & travel books. Whether you're interested in traveling to a new city, going on a cruise, or cooking a new dish — we're committed to inspiring you to experience travel in a whole new way. Lonely Planet's collection of 825+ travel and guidebooks is sure to inspire the traveler within. View All Books. Find out which places made the ultimate ranking of travel destinations based on

```json
{
    "action": "Final Answer",
    "action_input": "According to a search, some of the best places to travel include Paris, Glacier National Park, and many others. "
}
```

> Finished chain.
AI: According to a search, some of the best places to travel include Paris, Glacier National Park, and many others. 
